# 피벗테이블로 데이터 집계하기
- 피벗테이블은 표의 데이터를 요약하는 통계표이다.
- pd.pivot_table(데이터프레임, index=인덱스, columns=컬럼, values=집계할데이터, aggfunc=통계함수)
- aggrunc의 디폴트는 mean

In [3]:
import pandas as pd

In [4]:
# 샘플데이터
df = pd.DataFrame(
        {"item": ['shirts','shirts','shirts','shirts','shirts',
                 'pants','pants','pants','pants'],
         'color': ['white','white','white','black','black',
                  'white','white','black','black'],
         'size': ['small','large','large','small',
                 'small','large','small','small',
                 'large'],
         'sale': [1,2,2,3,3,4,5,6,7],
         'inventory': [2,4,5,5,6,6,8,9,9]
        }
)

In [5]:
df

,item,color,size,sale,inventory
0,shirts,white,small,1,2
1,shirts,white,large,2,4
2,shirts,white,large,2,5
3,shirts,black,small,3,5
4,shirts,black,small,3,6
5,pants,white,large,4,6
6,pants,white,small,5,8
7,pants,black,small,6,9
8,pants,black,large,7,9


In [10]:
# item, size별 재고 합계
df.pivot_table(index='item', columns='size', values='inventory', aggfunc='sum')

size,large,small
item,,
pants,15,17
shirts,9,13


In [11]:
# [item,color], size별 재고 합계
df.pivot_table(index=['item','color'], columns='size', values='inventory', aggfunc='sum')

size          large  small
item   color              
pants  black    9.0    9.0
       white    6.0    8.0
shirts black    NaN   11.0
       white    9.0    2.0

In [12]:
# null값은 0으로 처리
df.pivot_table(index=['item','color'], columns='size', values='inventory', aggfunc='sum', fill_value=0)

size          large  small
item   color              
pants  black      9      9
       white      6      8
shirts black      0     11
       white      9      2

In [13]:
# [item,color], size별 판매,재고 합계
df.pivot_table(index=['item','color'], columns='size', values=['sale','inventory'], aggfunc='sum', fill_value=0)

inventory        sale      
size             large small large small
item   color                            
pants  black         9     9     7     6
       white         6     8     4     5
shirts black         0    11     0     6
       white         9     2     4     1

# 타이타닉호 성별, 객실등급별 생존분석

In [15]:
import pandas as pd
df = pd.read_csv('data/titanic.csv')
df_titanic = df[['Survived','Pclass','Sex','Age','Embarked']]
df_titanic = df_titanic.dropna()
df_titanic.head()

,Survived,Pclass,Sex,Age,Embarked
0,0,3,male,22.0,S
1,1,1,female,38.0,C
2,1,3,female,26.0,S
3,1,1,female,35.0,S
4,0,3,male,35.0,S


In [71]:
len(df_titanic)

1044

## 성별, 객실등급별 승선자 수
- count

In [21]:
df_titanic.pivot_table(index='Sex',columns='Pclass',values='Survived',aggfunc='count', margins=True) # margins=True -> 가로,세로줄 합계

Pclass,1,2,3,All
Sex,,,,
female,131,103,152,386
male,151,158,349,658
All,282,261,501,1044


## 성별, 객실등급별
생존자 수
- sum

In [22]:
# 생존자들만을 조사하면.
df_titanic.pivot_table(index='Sex',columns='Pclass',values='Survived',aggfunc='sum', margins=True)

Pclass,1,2,3,All
Sex,,,,
female,128,97,97,322
male,40,15,38,93
All,168,112,135,415


## 성별, 객실등급별 생존율
- mean(default)

In [24]:
df_titanic.pivot_table(index='Sex',columns='Pclass',values='Survived',aggfunc='mean', margins=True)

Pclass,1,2,3,All
Sex,,,,
female,0.977099,0.941748,0.638158,0.834197
male,0.264901,0.094937,0.108883,0.141337
All,0.595745,0.429119,0.269461,0.397510


In [25]:
df_titanic.pivot_table(index='Sex',columns='Pclass',values='Survived', margins=True)

Pclass,1,2,3,All
Sex,,,,
female,0.977099,0.941748,0.638158,0.834197
male,0.264901,0.094937,0.108883,0.141337
All,0.595745,0.429119,0.269461,0.397510


In [26]:
##

In [27]:
# 기준 컬럼에 따라 데이터프레임을 그룹화하고, 그룹에 따른 통계값을 집계할 수 있다.

In [28]:
import pandas as pd
df = pd.read_csv('data/titanic.csv')
df = df[['Survived','Pclass','Sex','Age','Embarked']]
df = df_titanic.dropna()
df.head()

,Survived,Pclass,Sex,Age,Embarked
0,0,3,male,22.0,S
1,1,1,female,38.0,C
2,1,3,female,26.0,S
3,1,1,female,35.0,S
4,0,3,male,35.0,S


In [29]:
len(df)

1044

# 그룹의 통계값 계산하기 (groupby)
- 데이터프레임.groupby(그룹기준컬럼).통계적용컬럼.통계함수
- count() : 누락값을 제외한 데이터 수
- size() : 누락값을 포함한 데이터 수
- mean() : 평균
- sum() : 합계
- std() : 표준편차
- min() : 최소값
- max() : 최대값
- sum() : 전체합

## 객실등급별 생존 통계

In [36]:
# 객실등급(Pclass)별 승선자 수를 구한 결과를 데이터프레임 df1로 만들기
df1 = df.groupby('Pclass').Survived.count().to_frame() # to_frame() -> 데이터프레임화
df1

,Survived
Pclass,
1,282
2,261
3,501


In [37]:
# 객실등급(Pclass)별 생존자 수를 구한 결과를 데이터프레임 df2로 만들기
df2 = df.groupby('Pclass').Survived.sum().to_frame()
df2

,Survived
Pclass,
1,168
2,112
3,135


In [38]:
# 객실등급(Pclass)별 생존율을 구한 결과를 데이터프레임 df3로 만들기
df3 = df.groupby('Pclass').Survived.mean().to_frame()
df3

,Survived
Pclass,
1,0.595745
2,0.429119
3,0.269461


In [41]:
# 객실등급(Pclass)별 승선자수, 생존자수, 생존율 데이터프레임을 df4로 만들기
df4 = pd.concat([df1,df2,df3], axis=1) # concat -> 데이터 프레임 합치기
df4.columns=['승선자수','생존자수','생존율']
df4

,승선자수,생존자수,생존율
Pclass,,,
1,282,168,0.595745
2,261,112,0.429119
3,501,135,0.269461


## 성별 생존 통계

In [45]:
# 성별 승선자 수 데이터프레임을 df5로 만들기
df5 = df.groupby('Sex').Survived.count().to_frame()
df5

,Survived
Sex,
female,386
male,658


In [46]:
# 성별 생존자 수 데이터프레임을 df6으로 만들기
df6 = df.groupby('Sex').Survived.sum().to_frame()
df6

,Survived
Sex,
female,322
male,93


In [47]:
# 성별 생존율 데이터프레임을 df7로 만들기
df7 = df.groupby('Sex').Survived.mean().to_frame()
df7

,Survived
Sex,
female,0.834197
male,0.141337


In [49]:
# 성별 승선자수, 생존자수, 생존율 데이터프레임을 df8로 만들기
df8 = pd.concat([df5,df6,df7], axis=1)
df8.columns=['승선자수','생존자수','생존율']
df8

,승선자수,생존자수,생존율
Sex,,,
female,386,322,0.834197
male,658,93,0.141337


## 성별, 객실등급별 생존 통계

In [51]:
# 성별, 객실등급별 생존 통계
df.groupby(['Sex','Pclass']).Survived.mean().to_frame()

Survived
Sex    Pclass          
female 1       0.977099
       2       0.941748
       3       0.638158
male   1       0.264901
       2       0.094937
       3       0.108883

# 그룹에 사용자 정의함수 적용하기
- 데이터프레임.groupby(그룹기준컬럼).통계적용컬럼.agg(사용자정의함수,매개변수들)

In [52]:
def my_mean(values):
    return sum(values)/len(values)

In [54]:
df.groupby(['Sex','Pclass']).Survived.agg(my_mean)

Sex     Pclass
female  1         0.977099
        2         0.941748
        3         0.638158
male    1         0.264901
        2         0.094937
        3         0.108883
Name: Survived, dtype: float64

# 그룹 오브젝트 출력하기
- 데이터프레임.groupby(그룹기준컬럼).groups -> 그룹별 인덱스:[데이터리스트]출력
- 데이터프레임.groupby(그룹기준컬럼).get_group(그룹인덱스) -> 그룹별 인덱스에 해당하는 데이터프레임 출력

In [55]:
df20 = df[:20]
df20.head()

,Survived,Pclass,Sex,Age,Embarked
0,0,3,male,22.0,S
1,1,1,female,38.0,C
2,1,3,female,26.0,S
3,1,1,female,35.0,S
4,0,3,male,35.0,S


In [56]:
len(df20)

20

In [58]:
# Pclass 그룹별 인덱스
df20.groupby('Pclass').groups

In [65]:
# Pclass 그룹 출력(1등석)
df20.groupby('Pclass').get_group(1)

,Survived,Pclass,Sex,Age,Embarked
1,1,1,female,38.0,C
3,1,1,female,35.0,S
6,0,1,male,54.0,S
11,1,1,female,58.0,S


In [66]:
# Pclass 그룹 출력(2등석)
df20.groupby('Pclass').get_group(2)

,Survived,Pclass,Sex,Age,Embarked
9,1,2,female,14.0,C
15,1,2,female,55.0,S
20,0,2,male,35.0,S
21,1,2,male,34.0,S


In [67]:
# Pclass 그룹 출력(3등석)
df20.groupby('Pclass').get_group(3)

,Survived,Pclass,Sex,Age,Embarked
0,0,3,male,22.0,S
2,1,3,female,26.0,S
4,0,3,male,35.0,S
7,0,3,male,2.0,S
8,1,3,female,27.0,S
10,1,3,female,4.0,S
12,0,3,male,20.0,S
13,0,3,male,39.0,S
14,0,3,female,14.0,S
16,0,3,male,2.0,Q
